# Connection with local DataBase

In [59]:
from sqlalchemy import create_engine
import urllib
import pandas as pd

# Your existing pyodbc connection details
server = r"MELKON\SQLEXPRESS"
database = "DataWarehouse"

# Construct ODBC connection string
odbc_str = f"DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;TrustServerCertificate=yes;"

# Encode for SQLAlchemy
connection_url = f"mssql+pyodbc:///?odbc_connect={urllib.parse.quote_plus(odbc_str)}"

# Create SQLAlchemy engine
engine = create_engine(connection_url)

**Script Overview:**

======================================================================================================================
This SQL script updates the `silver.crm_cust_info` table by adding an `is_future` column and inserting the latest 
cleaned customer records from `bronze.crm_cust_info`. It ensures data consistency by:  

- Selecting the most recent record per customer (`cust_id`) using `ROW_NUMBER()`.  
- Cleaning names and standardizing marital status and gender values.  
- Flagging future-dated records in the `is_future` column.  

This script is part of an ETL process, ensuring accurate and up-to-date customer data for analytics and reporting. 🚀

=======================================================================================================================

In [58]:
# SQL Query
query = """
SELECT
    cust_id,
    cust_key,
    TRIM(cust_firstname) AS cust_firstname,
    TRIM(cust_lastname) AS cust_lastname,
    CASE 
        WHEN UPPER(TRIM(cust_marital_status)) = 'M' THEN 'Married'
        WHEN UPPER(TRIM(cust_marital_status)) = 'S' THEN 'Single'
        ELSE 'n/a'
    END AS cust_marital_status,
    CASE 
        WHEN UPPER(TRIM(cust_gender)) = 'M' THEN 'Male'
        WHEN UPPER(TRIM(cust_gender)) = 'F' THEN 'Female'
        ELSE 'n/a'
    END AS cust_gender,
    cust_create_date,
    CASE 
        WHEN cust_create_date IS NULL THEN 0
        WHEN cust_create_date > GETDATE() THEN 1
        ELSE 0
    END AS cust_is_future
FROM (
    SELECT
        *,
        ROW_NUMBER() OVER(PARTITION BY cust_id ORDER BY cust_create_date DESC) AS flag_last
    FROM bronze.crm_cust_info
    WHERE cust_id IS NOT NULL
) latest_record
WHERE flag_last = 1;
"""

# Execute query using SQLAlchemy engine
df_results = pd.read_sql(query, engine)

# Display DataFrame
df_results

,cust_id,cust_key,cust_firstname,cust_lastname,cust_marital_status,cust_gender,cust_create_date,cust_is_future
0,11000,AW00011000,Jon,Yang,Married,Male,2025-10-06,1
1,11001,AW00011001,Eugene,Huang,Single,Male,2025-10-06,1
2,11002,AW00011002,Ruben,Torres,Married,Male,2025-10-06,1
3,11003,AW00011003,Christy,Zhu,Single,Female,2025-10-06,1
4,11004,AW00011004,Elizabeth,Johnson,Single,Female,2025-10-06,1
...,...,...,...,...,...,...,...,...
18479,29479,AW00029479,Tommy,Tang,Married,n/a,2026-01-25,1
18480,29480,AW00029480,Nina,Raji,Single,n/a,2026-01-25,1
18481,29481,AW00029481,Ivan,Suri,Single,n/a,2026-01-25,1
18482,29482,AW00029482,Clayton,Zhang,Married,n/a,2026-01-25,1


In [60]:
df = df_results
print(df.head())

   cust_id     cust_key cust_firstname cust_lastname cust_marital_status  \
0   11000  AW00011000           Jon         Yang             Married   
1   11001  AW00011001        Eugene        Huang              Single   
2   11002  AW00011002         Ruben       Torres             Married   
3   11003  AW00011003       Christy          Zhu              Single   
4   11004  AW00011004     Elizabeth      Johnson              Single   

  cust_gender cust_create_date  cust_is_future  
0       Male      2025-10-06              1  
1       Male      2025-10-06              1  
2       Male      2025-10-06              1  
3     Female      2025-10-06              1  
4     Female      2025-10-06              1  


In [4]:
import pandas as pd
from sqlalchemy import create_engine
import urllib

# Database Connection
server = r"MELKON\SQLEXPRESS"
database = "DataWarehouse"

# Construct ODBC connection string
odbc_str = f"DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;TrustServerCertificate=yes;"
connection_url = f"mssql+pyodbc:///?odbc_connect={urllib.parse.quote_plus(odbc_str)}"

# Create SQLAlchemy engine
engine = create_engine(connection_url)

# Step 1: Get all tables in 'bronze' schema
query_get_tables = """
SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES
WHERE TABLE_SCHEMA = 'bronze'
"""
df_tables = pd.read_sql(query_get_tables, engine)
table_names = df_tables['TABLE_NAME'].tolist()

# Define common primary key names
possible_primary_keys = ['cust_id', 'id', 'cid', 'prd_id', 'sls_id', 'order_id']

# Step 2: Iterate through each table
for table_name in table_names:
    print(f"\n🔹 Processing Table: {table_name}\n")

    # Step 3: Get all column names and data types for the current table
    query_get_columns = f"""
    SELECT COLUMN_NAME, DATA_TYPE 
    FROM INFORMATION_SCHEMA.COLUMNS
    WHERE TABLE_SCHEMA = 'bronze' AND TABLE_NAME = '{table_name}'
    """
    df_columns = pd.read_sql(query_get_columns, engine)
    column_names = df_columns['COLUMN_NAME'].tolist()
    column_data_types = dict(zip(df_columns['COLUMN_NAME'], df_columns['DATA_TYPE']))

    # Identify a primary key if available
    primary_key = next((col for col in column_names if col in possible_primary_keys), None)

    # Step 4: Generate dynamic SELECT statement with cleansing rules
    select_clauses = []
    
    for col in column_names:
        col_lower = col.lower()
        data_type = column_data_types[col]
        
        if col_lower == 'cust_firstname' or col_lower == 'cust_lastname':
            # Trim name fields
            select_clauses.append(f"TRIM({col}) AS {col}")
        elif col_lower == 'cust_marital_status':
            # Standardize marital_status
            select_clauses.append(f"""
                CASE 
                    WHEN UPPER(TRIM({col})) = 'M' THEN 'Married'
                    WHEN UPPER(TRIM({col})) = 'S' THEN 'Single'
                    ELSE 'n/a'
                END AS {col}
            """)
        elif col_lower == 'cust_gender':
            # Standardize gender
            select_clauses.append(f"""
                CASE 
                    WHEN UPPER(TRIM({col})) = 'M' THEN 'Male'
                    WHEN UPPER(TRIM({col})) = 'F' THEN 'Female'
                    ELSE 'n/a'
                END AS {col}
            """)
        elif col_lower == 'cust_create_date':
            # Preserve the raw create_date
            select_clauses.append(f"{col}")
            # And add a computed column for cust_is_future
            select_clauses.append(f"""
                CASE 
                    WHEN {col} IS NULL THEN 0
                    WHEN {col} > GETDATE() THEN 1
                    ELSE 0
                END AS cust_is_future
            """)
        elif data_type in ['varchar', 'nvarchar']:
            # For other text columns, trim the value
            select_clauses.append(f"TRIM({col}) AS {col}")
        else:
            # Otherwise, select the column as-is
            select_clauses.append(f"{col}")
    
    # Build the SELECT clause string
    select_query = ",\n    ".join(select_clauses)

    # Step 5: Construct the final query
    # If the table has a primary key and a raw create_date column, use the ROW_NUMBER() filtering.
    if primary_key and ('cust_create_date' in column_names):
        query_clean = f"""
        SELECT
            {select_query}
        FROM (
            SELECT
                *,
                ROW_NUMBER() OVER(PARTITION BY {primary_key} ORDER BY cust_create_date DESC) AS flag_last
            FROM bronze.{table_name}
        ) latest_record
        WHERE flag_last = 1;
        """
    else:
        # Otherwise, do a simple select
        query_clean = f"""
        SELECT {select_query} FROM bronze.{table_name};
        """
    
    # Display the generated query for verification
    print(f"Generated Query:\n{query_clean}\n")
    
    # Step 6: Execute the query and load results into a DataFrame
    df_cleaned = pd.read_sql(query_clean, engine)
    
    # Display the cleaned data for this table
    print(df_cleaned)

print("\n✅ Cleaning process completed for all tables in 'bronze' schema!")



🔹 Processing Table: crm_cust_info

Generated Query:

        SELECT
            cust_id,
    TRIM(cust_key) AS cust_key,
    TRIM(cust_firstname) AS cust_firstname,
    TRIM(cust_lastname) AS cust_lastname,
    
                CASE 
                    WHEN UPPER(TRIM(cust_marital_status)) = 'M' THEN 'Married'
                    WHEN UPPER(TRIM(cust_marital_status)) = 'S' THEN 'Single'
                    ELSE 'n/a'
                END AS cust_marital_status
            ,
    
                CASE 
                    WHEN UPPER(TRIM(cust_gender)) = 'M' THEN 'Male'
                    WHEN UPPER(TRIM(cust_gender)) = 'F' THEN 'Female'
                    ELSE 'n/a'
                END AS cust_gender
            ,
    cust_create_date,
    
                CASE 
                    WHEN cust_create_date IS NULL THEN 0
                    WHEN cust_create_date > GETDATE() THEN 1
                    ELSE 0
                END AS cust_is_future
            
        FROM (
            SEL

In [5]:
print(df_cleaned)


       id          cat             subcat maintenance
0   AC_BR  Accessories         Bike Racks         Yes
1   AC_BS  Accessories        Bike Stands          No
2   AC_BC  Accessories  Bottles and Cages          No
3   AC_CL  Accessories           Cleaners         Yes
4   AC_FE  Accessories            Fenders          No
5   AC_HE  Accessories            Helmets         Yes
6   AC_HP  Accessories    Hydration Packs          No
7   AC_LI  Accessories             Lights         Yes
8   AC_LO  Accessories              Locks         Yes
9   AC_PA  Accessories           Panniers          No
10  AC_PU  Accessories              Pumps         Yes
11  AC_TT  Accessories    Tires and Tubes         Yes
12  BI_MB        Bikes     Mountain Bikes         Yes
13  BI_RB        Bikes         Road Bikes         Yes
14  BI_TB        Bikes      Touring Bikes         Yes
15  CL_BS     Clothing         Bib-Shorts          No
16  CL_CA     Clothing               Caps          No
17  CL_GL     Clothing      